from Collections import Counter

In [1]:
import torch
import numpy as np
import warnings
warnings.filterwarnings("ignore")
def normalized_laplacian(adj_matrix):
    R = np.sum(adj_matrix, axis=1)
    adj_matrix = adj_matrix + np.eye(adj_matrix.shape[0])
    R_sqrt = 1/np.sqrt(R)
    D_sqrt = np.diag(R_sqrt) + np.eye(adj_matrix.shape[0])
    return np.matmul(np.matmul(D_sqrt, adj_matrix), D_sqrt)

In [2]:
import glob
# graph
batch = 0
data_dir = glob.glob('dataset/SC_Full_S*_data.pth')
all_label = []
all_data = []
all_data_graph = []
batch =0
flag = 0
data_dir = sorted(data_dir)
for index in data_dir:
    data = torch.load(index)
    data = torch.stack(data)
    data += 1e-3*torch.eye(data.shape[-1])
    data =  data
    if flag == 0:
        all_data_graph = data
        flag = 1
    else:
        all_data_graph = torch.cat((all_data_graph, data),dim =0)
    batch+=1
    print(index)

# data
batch = 0
data_dir = glob.glob('dataset/FC_Full_S*_data.pth')
all_label = []
all_data = []
batch =0
flag = 0
data_dir = sorted(data_dir)
for index in data_dir:
    data = torch.load(index)
    data = torch.stack(data)
    data += 1e-3*torch.eye(data.shape[-1])
    if flag == 0:
        all_data = data
        flag = 1
    else:
        all_data = torch.cat((all_data, data),dim =0)
    batch += 1

batch = 0
all_label_dir = glob.glob('dataset/labels_Full_S*_1.pth')
all_label_dir = sorted(all_label_dir)
flag = 0
for index in all_label_dir:
    data = torch.load(index)
    data = torch.stack(data)
    # data += 1e-3*torch.eye(data.shape[-1])
    if flag == 0:
        all_label = data
        flag = 1
    else:
        all_label = torch.cat((all_label,data),dim =0)
    batch += 1
    print(index)


dataset/SC_Full_S13_data.pth
dataset/labels_Full_S13_1.pth


In [3]:
import bisect
import math
import torch
class LRScheduler:
    def __init__(self):
        self.lr_config = 0
        self.training_config =0
        self.lr = 0

    def update(self, optimizer: torch.optim.Optimizer, step: int):
        lr_mode = 'cos'
        base_lr = 1.0e-4
        target_lr = 1.0e-6

        warm_up_from = 0.0
        warm_up_steps = 0
        # total_steps = self.training_config.total_steps
        total_steps = 11200
        milestones = [0.3, 0.6, 0.9]
        lr_decay = 0.98
        decay_factor = 0.1
        poly_power = 2.0
        assert 0 <= step <= total_steps
        if step < warm_up_steps:
            current_ratio = step / warm_up_steps
            self.lr = warm_up_from + (base_lr - warm_up_from) * current_ratio
        else:
            current_ratio = (step - warm_up_steps) / \
                (total_steps - warm_up_steps)
            if lr_mode == 'step':
                count = bisect.bisect_left(milestones, current_ratio)
                self.lr = base_lr * pow(decay_factor, count)
            elif lr_mode == 'poly':
                poly = pow(1 - current_ratio, poly_power)
                self.lr = target_lr + (base_lr - target_lr) * poly
            elif lr_mode == 'cos':
                cosine = math.cos(math.pi * current_ratio)
                self.lr = target_lr + (base_lr - target_lr) * (1 + cosine) / 2
            elif lr_mode == 'linear':
                self.lr = target_lr + \
                    (base_lr - target_lr) * (1 - current_ratio)
            elif lr_mode == 'decay':
                epoch = step // self.training_config.steps_per_epoch
                self.lr = base_lr * lr_decay ** epoch

        for param_group in optimizer.param_groups:
            param_group['lr'] = self.lr


In [7]:
from sklearn.model_selection import KFold,StratifiedKFold
from torch.utils.data import DataLoader, Subset
import torch.nn.functional as F
from torch import nn
from torch.optim import Adam,SGD
from torch.autograd import Variable
from sklearn.metrics import precision_score, recall_score
import copy
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from model.gdotnet import CRATE_edgev3
from sklearn.model_selection import train_test_split
device = 'cuda'
best_state = 0
best_state_acc = 0
best_f1_score = 0
best_auc = 0
fold = 5
res_acc = []
res_f1_score = []
res_auc = []
res_pre = []
res_rec = []
result_acc = 0
result_auc = 0
result_f1_score = 0
result_recall = 0
result_pre = 0

torch.manual_seed(49)
# kf = StratifiedKFold(n_splits=fold,shuffle=True,random_state=70) #70
    # kf = KFold(n_splits=fold,shuffle=True,random_state=0)
fold_no = 1
result_acc = 0
result_auc = 0
result_f1_score = 0
result_recall = 0
result_pre = 0
iters = 5
for i in range(iters):
    kf = StratifiedKFold(n_splits=fold,shuffle=True, random_state=70) #70
    for train_index, val_index in kf.split(all_data_graph, all_label):
        # train_subset, train_graph, train_label = all_data[train_index], all_data_graph[train_index], all_label[train_index]
        # val_subset, val_graph, val_label = all_data[val_index], all_data_graph[val_index], all_label[val_index]
        train_ratio = 0.7
        val_ratio = 0.1
        test_ratio = 0.2
        indices = np.arange(len(all_label))
        train_index, temp_index, train_label_split, temp_label_split = train_test_split(
            indices, 
            all_label,
            test_size=(val_ratio + test_ratio), # 0.1 + 0.2 = 0.3
            shuffle=True,
            stratify=all_label,# for reproducible results
        )

        # Split 2: Split the 30% "Temp" set into Validation (10%) and Test (20%)
        # The new test_size is relative to the temp set: 
        # test_ratio / (val_ratio + test_ratio) = 0.2 / 0.3 = 0.666...
        val_index, test_index = train_test_split(
            temp_index,
            test_size=(test_ratio / (val_ratio + test_ratio)),
            shuffle=True,
            stratify=temp_label_split, # Stratify based on the temp set's labels
            random_state=42 # for reproducible results
        )


        # Training set (70%)
        train_subset, train_graph, train_label = all_data[train_index], all_data_graph[train_index], all_label[train_index]

        # Validation set (10%)
        val_subset, val_graph, val_label = all_data[val_index], all_data_graph[val_index], all_label[val_index]

        # Test set (20%)
        test_subset, test_graph, test_label = all_data[test_index], all_data_graph[test_index], all_label[test_index]
        train_label = train_label.squeeze(1)
        val_label = val_label.squeeze(1)
        test_label = test_label.squeeze(1)
        train_label = F.one_hot(train_label.to(torch.int64))
        val_label = F.one_hot(val_label.to(torch.int64))
        test_label = F.one_hot(test_label.to(torch.int64))
            # 创建DataLoader

        train_set = torch.utils.data.TensorDataset(train_graph, train_subset,  train_label)
        test_set = torch.utils.data.TensorDataset(test_graph, test_subset, test_label)
        # val_set = torch.utils.data.TensorDataset(val_graph, val_subset, val_label)
        train_loader = DataLoader(train_set, batch_size=512, num_workers=0, shuffle=True)
        val_loader = DataLoader(test_set, batch_size=512, num_workers=0, shuffle=False)
        # test_loader = DataLoader(test_set, batch_size=512, num_workers=0, shuffle=True)

        loss_func = torch.nn.CrossEntropyLoss(reduction='sum')

        lr = 0.0001
        lr_scheduler= LRScheduler()
        current_step = 0

        epochs = 50
        flag_epoch = 0
        best_accuray = 0
        best_model = 0
        best_pre = 0
        best_rec = 0
        g_model = CRATE_edgev3()
        # g_model = subgraphkan()
        g_model = g_model.to(device)

        g_optimizer = Adam(g_model.parameters(), lr=lr, weight_decay=0.02)
        for epoch in range(epochs):
            prob_all = []
            label_all = []
            auc_all = []
            total_loss = 0
            accuracy = 0
            g_model.train()
            for batch_idx, (train_graph, train_data, training_labels) in enumerate(train_loader):
                current_step += 1
                lr_scheduler.update(optimizer=g_optimizer, step=current_step)
                train_data, train_graph, training_labels = train_data.to(device), train_graph.to(device), training_labels.to(device)
                training_labels = training_labels.float()
                g_optimizer.zero_grad()
                pred_label ,output= g_model(train_graph, train_data, training_labels[:,1].long())
                
                loss = loss_func(pred_label, training_labels)
                loss.backward()
                g_optimizer.step() 
                training_labels = torch.max(training_labels, 1)[1].squeeze()
                pred_y = torch.max(pred_label,1)[1].squeeze() #squeeze()默认是将a中所有为1的维度删掉
                            #pred_size() = [10000]
                accuracy += sum(pred_y == training_labels) / train_data.__len__()
            # print('Eopch:', int(epoch), ' | train loss: %.6f' % loss.item(), ' | accracy:%.5f' % (accuracy/(batch_idx+1)))
            g_model.eval()
            accuracy = 0
            result = []
            labels = []
            with torch.no_grad():
                for batch_idx, (train_graph, train_data, training_labels) in enumerate(val_loader):
                    train_data, training_labels = train_data.to(device), training_labels.to(device)
                    train_graph = train_graph.to(device)
                    pred_label,_ = g_model(train_graph, train_data, None)
                    
                    # training_labels = training_labels.float()
                    result += F.softmax(pred_label, dim=1)[:, 1].tolist()
                    labels += training_labels[:, 1].tolist()
                    pred_y = torch.max(pred_label,1)[1].squeeze() #squeeze()默认是将a中所有为1的维度删掉
                                #pred_size() = [10000]
                    pred_label = F.softmax(pred_label,dim=1)
                    training_labels = training_labels[:,1]
                    accuracy += sum(pred_y == training_labels) / train_data.__len__()
                    prob_all.extend(pred_y.cpu().numpy())
                    label_all.extend(training_labels.cpu().numpy())
                    auc_all.extend(pred_label[:, 1].detach().cpu().numpy())
            auc = roc_auc_score(labels, result,average='weighted')
            result, labels = np.array(result), np.array(labels)
            result[result >= 0.5] = 1
            result[result < 0.5] = 0
            f1 = f1_score(labels, result, average='weighted')
            rec = recall_score(labels, result, average='weighted')
            pre = precision_score(labels, result, average='weighted')
            accuracy = accuracy/(batch_idx+1)
            if accuracy>best_accuray:
                # print(accuracy,pre,rec)
                best_accuray = accuracy 
                    # best_model = copy.deepcopy(g_model)
                best_auc = auc
                best_f1_score = f1
                best_pre = pre
                best_rec = rec
        # break

            print('Eopch:', int(epoch))
            print('Precision: %.6f'%(pre), 'recall: %.6f'%(rec),' | accracy:%.5f' % (accuracy), ' | f1_score:%.5f' % (f1), ' | auc:%.5f' % (auc))
            # print("****************************************")
        break
    

    res_acc.append(best_accuray.cpu().numpy())
    res_auc.append(best_auc)
    res_f1_score.append(best_f1_score)
    res_pre.append(best_pre)
    res_rec.append(best_rec)
        # print('best_Precision: %.6f'%(best_pre), 'best_recall: %.6f'%(best_rec),' | accracy:%.5f' % (best_accuray), ' | f1_score:%.5f' % (best_f1_score), ' | auc:%.5f' % (best_auc))
    # break

Eopch: 0
Precision: 0.444444 recall: 0.666667  | accracy:0.66667  | f1_score:0.53333  | auc:0.44444
Eopch: 1
Precision: 0.444444 recall: 0.666667  | accracy:0.66667  | f1_score:0.53333  | auc:0.22222
Eopch: 2
Precision: 0.444444 recall: 0.666667  | accracy:0.66667  | f1_score:0.53333  | auc:0.22222
Eopch: 3
Precision: 0.444444 recall: 0.666667  | accracy:0.66667  | f1_score:0.53333  | auc:0.27778
Eopch: 4
Precision: 0.444444 recall: 0.666667  | accracy:0.66667  | f1_score:0.53333  | auc:0.33333
Eopch: 5
Precision: 0.444444 recall: 0.666667  | accracy:0.66667  | f1_score:0.53333  | auc:0.33333
Eopch: 6
Precision: 0.444444 recall: 0.666667  | accracy:0.66667  | f1_score:0.53333  | auc:0.44444
Eopch: 7
Precision: 0.444444 recall: 0.666667  | accracy:0.66667  | f1_score:0.53333  | auc:0.44444
Eopch: 8
Precision: 0.444444 recall: 0.666667  | accracy:0.66667  | f1_score:0.53333  | auc:0.44444
Eopch: 9
Precision: 0.444444 recall: 0.666667  | accracy:0.66667  | f1_score:0.53333  | auc:0.44444


In [8]:
acc_f = np.mean(res_acc)
f1_f = np.mean(res_f1_score)
auc_f = np.mean(res_auc)
pre_f = np.mean(res_pre)
rec_f = np.mean(res_rec)
print("acc:", acc_f)
print("f1:", f1_f)
print("auc:", auc_f)
print("pre:", pre_f)
print("rec:", rec_f)

acc: 0.8
f1: 0.7841269841269841
auc: 0.7111111111111111
pre: 0.8555555555555555
rec: 0.7999999999999999
